In [1]:
import streamlit as st
import os
from langchain.tools import DuckDuckGoSearchRun
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_agent_executor
from langchain_core.pydantic_v1 import BaseModel
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Annotated, Any, Dict, Optional, Sequence, TypedDict, List, Tuple
from dotenv import load_dotenv, dotenv_values 
import operator

load_dotenv()

True

In [2]:
langchain_api_key = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_PROJECT"] ="multi-agent"
os.environ['OPENAI_API_KEY'] = langchain_api_key

In [3]:
llm = ChatOpenAI(temperature=0, 
                          max_tokens=1024,
    model_name="gpt-3.5-turbo-0125")

In [4]:
class DataScraper(BaseModel):
    """This Class is for Data Scraping"""
    
    code: str = Field(
        description = "Generate code based on the provided requirements"
    )
    
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate

data_scrape_prompt = ChatPromptTemplate.from_template(
    '''**Role**: You are an expert Python programmer with a focus on data manipulation and analysis. Your expertise extends to handling and processing data from various formats, including CSV files.

**Task**: Your task is to develop Python code that reads data from a CSV file located at './data.csv'. After reading the data, you must process it according to specific requirements to return a subset of the DataFrame. This task may involve filtering, aggregating, transforming, or any other form of data manipulation to achieve the desired output.

**Instructions**:
1. **Understand and Clarify**: Ensure you fully understand the requirement, including the structure of the data and the specific subset or transformation needed.
2. **Algorithm/Method Selection**: Choose the most efficient methods for reading the CSV file and processing the data. Consider using libraries such as pandas for efficient data manipulation.
3. **Pseudocode Creation**: Write down the steps to read the data from the CSV file, apply the necessary processing, and extract the required subset. This includes loading the data, inspecting its structure, applying specified transformations or filters, and selecting the relevant data to return.
4. **Code Generation**: Translate your pseudocode into executable Python code. Your code should be efficient, readable, and well-commented. Ensure to include necessary imports, read the CSV file into a DataFrame, perform the required data processing, and output the subset of data as text.

*REQUIREMENT*
{requirement}'''

)

coder = create_structured_output_runnable(
    DataScraper, llm, data_scrape_prompt
)

In [5]:
code_ = coder.invoke({'requirement': 'Get me all the data for the country of germany'})

Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')


Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')


In [6]:
print(code_.code)

import pandas as pd

# Read the CSV file
data = pd.read_csv('./data.csv')

# Filter data for the country of Germany
germany_data = data[data['Country'] == 'Germany']

# Display the subset of data
print(germany_data)


In [7]:
from io import StringIO
import sys
tmp = sys.stdout
my_result = StringIO()
sys.stdout = my_result
exec(code_.code)
sys.stdout = tmp

In [8]:
print(my_result.getvalue())

       TransactionNo       Date ProductNo                        ProductName  \
914           581494  12/9/2019     23084                 Rabbit Night Light   
915           581494  12/9/2019     21559  Strawberry Lunch Box With Cutlery   
916           581494  12/9/2019     21506           Fancy Font Birthday Card   
917           581494  12/9/2019     22716                 Card Circus Parade   
918           581494  12/9/2019     22556      Plasters In Tin Circus Parade   
...              ...        ...       ...                                ...   
536345       C536548  12/1/2018     22168       Organiser Wood Antique White   
536346       C536548  12/1/2018     21218             Red Spotty Biscuit Tin   
536347       C536548  12/1/2018     20957       Porcelain Hanging Bell Small   
536348       C536548  12/1/2018     22580       Advent Calendar Gingham Sack   
536349       C536548  12/1/2018     22767         Triple Photo Frame Cornice   

        Price  Quantity  CustomerNo  Co

str